In [1]:
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_iris 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np

In [2]:
# Load the TF-IDF data from CSV files
X_train = pd.read_csv('train_tfidf.csv')
X_test = pd.read_csv('test_tfidf.csv')

X_train.drop(X_train.columns[0], axis=1, inplace=True)
X_test.drop(X_test.columns[0], axis=1, inplace=True)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

# Loading the labels
pd_train = pd.read_csv('train_labels.csv').values.ravel()
pd_test = pd.read_csv('test_labels.csv').values.ravel()

pd_train = list(pd_train)
pd_test = list(pd_test)

y_train = [1 if x == "non-suicide" else 0 for x in pd_train]
y_test = [1 if x == "non-suicide" else 0 for x in pd_test]
y_train = np.array(y_train)
y_test = np.array(y_test)

In [3]:
estimator = [] 
#estimator.append(('LR', LogisticRegression(solver ='lbfgs', multi_class ='multinomial', max_iter = 200))) 
estimator.append(('SVC', SVC(probability = True))) 
#estimator.append(('DTC', DecisionTreeClassifier())) 

In [4]:
lr_best = LogisticRegression(max_iter=1000, C = 1, penalty = 'l2', solver = 'newton-cg')
dtclf_best = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=1, min_samples_split=5, splitter='best')
svc = SVC()

In [5]:
from sklearn.ensemble import StackingClassifier

# Create a list of classifiers
level0 = [('lr_best', lr_best), ('dtclf_best', dtclf_best)]
level1 = lr_best

# Create a stacking classifier object
stack_clf = StackingClassifier(estimators=level0, final_estimator=level1, cv=5, n_jobs=-1, passthrough=True)

# Train the classifier
stack_clf.fit(X_train, y_train)

# Predict the validation set
y_pred = stack_clf.predict(X_test)

# Calculate the accuracy
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.9156


In [6]:
# Create a voting classifier object
from sklearn.ensemble import VotingClassifier

# Create a list of classifiers
estimators = [('lr_best', lr_best), ('dtclf_best', dtclf_best),('svc',svc), ('stack_clf',stack_clf)]

# Create a voting classifier object
ensemble = VotingClassifier(estimators, voting='hard')

# Train the classifier
ensemble.fit(X_train, y_train)

# Predict the validation set
y_pred = ensemble.predict(X_test)

# Calculate the accuracy
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.9152
